In [ ]:
import sys
import os
import torch
from src.models import FashionMNISTModel
from datasets import IncrementalFashionMNIST
from src.eval import evaluate
from src.utils import add_heads


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.CrossEntropyLoss()
datamanager = IncrementalFashionMNIST(increment=2, batch_size=64, shuffle=True)
datamanager.next_task()  # Move to the next task to set current_class correctly
model = FashionMNISTModel(output_size=10)
model.to(device)
save_path = './experiments/fashion_mnist_incremental/model_after_task_5.pth'

model.load_state_dict(torch.load(save_path))
model.eval()

dataloader = datamanager.get_whole_set(mode='test')
evaluate(model, dataloader, criterion, device)

Test Loss: 5.1994, Accuracy: 19.91%


In [7]:
from src.train import normal_train
model_demo = FashionMNISTModel()
model_demo.to(device)
train_loader = datamanager.get_whole_set(mode='train')
test_loader = datamanager.get_whole_set(mode='test')
optimizer = torch.optim.SGD(model_demo.parameters(), lr=0.01, momentum=0.9)
normal_train(model_demo, train_loader, criterion, optimizer, device, epochs=5)
evaluate(model_demo, test_loader, criterion, device)


Epoch 1/5: 100%|██████████| 938/938 [00:05<00:00, 174.92it/s, loss=0.628]


Epoch [1/5], Loss: 0.6215


Epoch 2/5: 100%|██████████| 938/938 [00:05<00:00, 174.32it/s, loss=1.06]


Epoch [2/5], Loss: 1.0554


Epoch 3/5: 100%|██████████| 938/938 [00:05<00:00, 179.29it/s, loss=1.48]


Epoch [3/5], Loss: 1.4504


Epoch 4/5: 100%|██████████| 938/938 [00:05<00:00, 177.43it/s, loss=1.81]  


Epoch [4/5], Loss: 1.8147


Epoch 5/5: 100%|██████████| 938/938 [00:05<00:00, 177.41it/s, loss=2.18]  


Epoch [5/5], Loss: 2.1592
Test Loss: 0.3866, Accuracy: 85.94%
